# Configuring and Running Pre-Trade Compliance

In this example we demonstrate the configuration of several simple pre-trade rules, run them against a sample portfolio that has some orders placed against it, and show how the results can be used to identify breached orders, and pin down the cause of the breach.

## Imports

In [64]:
import lusid
import lusid.models as models
import lusid.api as la
import lusid.models as lm
from lusid.models.upsert_compliance_rule_request import UpsertComplianceRuleRequest
from lusid.models.reference_list_request import ReferenceListRequest
from lusid import ApiException
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon.utilities import create_scope_id
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
    format_holdings_response,
)

import helper_functions as hf
from collections import defaultdict
import pandas as pd
import numpy as np
import json
import openpyxl
import os
import datetime
from datetime import datetime, timedelta, time, date
import pytz

pd.set_option('display.max_columns', None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

print ('LUSID Environment Initialised')
print ('API Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
API Version:  0.0.1.0


In [65]:
# define some APIs
properties_api = api_factory.build(la.PropertyDefinitionsApi)
referencelist_api = api_factory.build(la.ReferenceListsApi)
compliance_api = api_factory.build(la.ComplianceApi)
aggregation_api = api_factory.build(la.AggregationApi)
configuration_recipe_api = api_factory.build(la.ConfigurationRecipeApi)
portfolio_groups_api = api_factory.build(la.PortfolioGroupsApi)
portfolios_api = api_factory.build(la.PortfoliosApi)
instruments_api = api_factory.build(la.InstrumentsApi)

In [66]:
# and some variables
scope='simpleCompliance'
portfolio_code='EQUITY_UK'
portfolio_group_code='EQUITY_UK_GROUP'

## 1. Create instruments, portfolio and transactions to work with

We'll need a minimally populated environment to run compliance checks on.

### 1.0 Load transaction data

In [67]:
df = pd.read_csv("data/equity_transactions.csv")

### 1.1 Load instruments

In [68]:
# Don't run this. It literally does what it says. It's just here if you need it.
#delete_all_current_instruments(instruments_api)

In [69]:
instrument_mapping = {
    "identifier_mapping": {
        "ClientInternal": "instrument_id",
        "Isin": "ISIN",
        "Sedol": "sedol",
    },
    "required": {"name": "name"},
}

result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=df,
    mapping_required=instrument_mapping["required"],
    mapping_optional={},
    file_type="instruments",
    identifier_mapping=instrument_mapping["identifier_mapping"],
    property_columns=["sector"],
)

succ, failed, errors = format_instruments_response(result)
pd.DataFrame(
    data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]
)

/Users/tim/Documents/code/sample-notebooks/.venv/lib/python3.12/site-packages/lusidtools/cocoon/cocoon.py:1720: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_frame = data_frame.applymap(cocoon.utilities.convert_cell_value_to_string)


,success,failed,errors
0,10,0,0


### 1.2 Create portfolio

In [70]:
# Don't do this either. Really.
hf.delete_all_current_portfolios(portfolios_api, scope)

Deleting:
Code: EQUITY_UK 
Scope: simpleCompliance
All scopes deleted


In [71]:
portfolio_mapping = {
    "required": {
        "code": "portfolio_code",
        "display_name": "portfolio_name",
        "base_currency": "$USD",
    },
    "optional": {"created": "$2020-01-01T00:00:00+00:00"},
}

result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=df,
    mapping_required=portfolio_mapping["required"],
    mapping_optional=portfolio_mapping["optional"],
    file_type="portfolios",
    sub_holding_keys=[],
)

succ, failed = format_portfolios_response(result)
pd.DataFrame(
    data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]
)

/Users/tim/Documents/code/sample-notebooks/.venv/lib/python3.12/site-packages/lusidtools/cocoon/cocoon.py:1720: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_frame = data_frame.applymap(cocoon.utilities.convert_cell_value_to_string)


,success,failed,errors
0,1,0,0


### 1.3 Create portfolio group

In [72]:
result = hf.create_portfolio_group(portfolio_groups_api, scope, portfolio_group_code, [lm.ResourceId(scope=scope, code=portfolio_code)])

### 1.4 Load transactions

In [73]:
transaction_mapping = {
    "identifier_mapping": {"ClientInternal": "instrument_id",},
    "required": {
        "code": "portfolio_code",
        "transaction_id": "txn_id",
        "type": "txn_type",
        "transaction_price.price": "txn_price",
        "transaction_price.type": "$Price",
        "total_consideration.amount": "txn_consideration",
        "units": "txn_units",
        "transaction_date": "txn_trade_date",
        "total_consideration.currency": "portfolio_base_currency",
        "settlement_date": "txn_settle_date",
    },
    "optional": {},
    "properties": [],
}

result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=df,
    mapping_required=transaction_mapping["required"],
    mapping_optional=transaction_mapping["optional"],
    file_type="transactions",
    identifier_mapping=transaction_mapping["identifier_mapping"],
    property_columns=transaction_mapping["properties"],
    properties_scope=scope,
)

succ, failed = format_transactions_response(result)
pd.DataFrame(
    data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]
)

/Users/tim/Documents/code/sample-notebooks/.venv/lib/python3.12/site-packages/lusidtools/cocoon/cocoon.py:1720: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_frame = data_frame.applymap(cocoon.utilities.convert_cell_value_to_string)


,success,failed,errors
0,1,0,0


### 1.5 Load quotes

In [74]:
quotes_df = pd.read_csv("data/equity_quotes.csv")
# Compliance runs at latest asat
quotes_df['quote_date'] = quotes_df['quote_date'].apply(lambda s: datetime.now(pytz.UTC).strftime("%d-%b-%y"))
quotes_df

,ISIN,quote_date,bid,mid,ask
0,GB0002162385,29-Jan-24,2.29,2.30,2.31
1,GB00BH0P3Z91,29-Jan-24,12.81,12.87,12.93
2,GB0031348658,29-Jan-24,0.88,1.88,0.88
3,GB0007980591,29-Jan-24,3.06,3.08,3.10
4,GB0005405286,29-Jan-24,4.00,4.02,4.04
5,GB0006043169,29-Jan-24,1.87,1.88,1.89
6,GB0008847096,29-Jan-24,2.35,2.36,2.37
7,GB00BGDT3G23,29-Jan-24,4.63,4.65,4.67
8,GB00BH4HKS39,29-Jan-24,1.08,1.09,1.10
9,GB00B1XZS820,29-Jan-24,13.90,13.97,14.04


In [75]:
price_fields = ["bid", "mid", "ask"]

quotes_mapping = {
    "quote_id.quote_series_id.instrument_id_type": "$Isin",
    "quote_id.effective_at": "quote_date",
    "quote_id.quote_series_id.provider": "$Lusid",
    "quote_id.quote_series_id.quote_type": "$Price",
    "quote_id.quote_series_id.instrument_id": "ISIN",
    "metric_value.unit": "$USD",
}

for price_field in price_fields:

    quotes_mapping["quote_id.quote_series_id.field"] = f"${price_field}"
    quotes_mapping["metric_value.value"] = price_field

    result = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=quotes_df,
        mapping_required=quotes_mapping,
        mapping_optional={},
        file_type="quotes",
    )

    succ, failed, errors = format_quotes_response(result)
    display(
        pd.DataFrame(
            data=[
                {
                    f"[{price_field}] success": len(succ),
                    "failed": len(failed),
                    "errors": len(errors),
                }
            ]
        )
    )

/Users/tim/Documents/code/sample-notebooks/.venv/lib/python3.12/site-packages/lusidtools/cocoon/cocoon.py:1720: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_frame = data_frame.applymap(cocoon.utilities.convert_cell_value_to_string)


,[bid] success,failed,errors
0,10,0,0


/Users/tim/Documents/code/sample-notebooks/.venv/lib/python3.12/site-packages/lusidtools/cocoon/cocoon.py:1720: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_frame = data_frame.applymap(cocoon.utilities.convert_cell_value_to_string)


,[mid] success,failed,errors
0,10,0,0


/Users/tim/Documents/code/sample-notebooks/.venv/lib/python3.12/site-packages/lusidtools/cocoon/cocoon.py:1720: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_frame = data_frame.applymap(cocoon.utilities.convert_cell_value_to_string)


,[ask] success,failed,errors
0,10,0,0


### 1.6 Create recipes

In [76]:
for price_field in ["mid", "bid", "ask"]:

    # Create a recipe to perform a valuation
    configuration_recipe = models.ConfigurationRecipe(
        scope="User",
        code="valuation_recipe" + "_" + price_field,
        market=models.MarketContext(
            market_rules=[
                # define how to resolve the quotes
                models.MarketDataKeyRule(
                    key="Quote.Isin.*",
                    supplier="Lusid",
                    data_scope=scope,
                    quote_type="Price",
                    field=price_field,
                ),
            ],
            options=models.MarketOptions(
                default_supplier="Lusid",
                default_instrument_code_type="Isin",
                default_scope=scope,
            ),
        ),
        pricing=models.PricingContext(
            options={"AllowPartiallySuccessfulEvaluation": True},
        ),
    )

    upsert_configuration_recipe_response = configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=models.UpsertRecipeRequest(
            configuration_recipe=configuration_recipe
        )
    )

### Run a test valuation

In [77]:
aggregation = aggregation_api.get_valuation(
    valuation_request = hf.generate_valuation_request(
        datetime.now(pytz.UTC), "mid", scope, portfolio_code
    )
)

pd.DataFrame(aggregation.data)

,Instrument/default/Name,Proportion(Valuation/PvInReportCcy),Sum(Valuation/PvInReportCcy),Sum(Holding/default/Units),Aggregation/Errors
0,Aviva,0.080752,276000.0,120000.0,[]
1,BHP,0.225929,772200.0,60000.0,[]
2,Barclays,0.082507,282000.0,150000.0,[]
3,BP,0.090114,308000.0,100000.0,[]
4,HSBC,0.023523,80400.0,20000.0,[]
5,Morrisons,0.099008,338400.0,180000.0,[]
6,Rightmove,0.108839,372000.0,80000.0,[]
7,vodafone,0.143510,490500.0,450000.0,[]
8,Anglo American plc,0.143056,488950.0,35000.0,[]
9,Tesco,0.002762,9440.0,4000.0,[]


## 2. Define properties and supporting data

In [78]:
try:
    hf.create_property_definition(properties_api, "Portfolio", scope, 'Placeholder', "string")
except:
    pass

### 2.1 Decorate entities with properties

In [79]:
response = portfolios_api.upsert_portfolio_properties(
        scope=scope,
        code=portfolio_code,
        request_body={
            "Portfolio/{}/Placeholder".format(
                scope
            ): models.ModelProperty(
                key="Portfolio/{}/Placeholder".format(scope),
                value=models.PropertyValue(label_value='PlaceholderValue'),
            )
        },
    )

### 2.2 Create reference lists

We need a couple of empty lists to allow pass-through behaviour.

In [80]:
request = ReferenceListRequest(
    id=lm.ResourceId(
        scope=scope,
        code='empty-list'
    ),
    name="empty string list",
    description="some description",
    tags=[],
    reference_list=lm.StringList(
        reference_list_type='StringList',
        values=[]
    )
)

empty_list_response = referencelist_api.upsert_reference_list_with_http_info(reference_list_request=request)

request = ReferenceListRequest(
    id=lm.ResourceId(
        scope=scope,
        code='empty-portfolio-list'
    ),
    name="empty portfolioid list",
    description="some description",
    tags=[],
    reference_list=lm.PortfolioIdList(
        reference_list_type='PortfolioIdList',
        values=[]
    )
)

empty_portfolioid_list_response = referencelist_api.upsert_reference_list_with_http_info(reference_list_request=request)

request = ReferenceListRequest(
    id=lm.ResourceId(
        scope=scope,
        code='restricted-sectors'
    ),
    name="restricted sectors list",
    description="some description",
    tags=[],
    reference_list=lm.StringList(
        reference_list_type='StringList',
        values=["Mining"]
    )
)

empty_portfolioid_list_response = referencelist_api.upsert_reference_list_with_http_info(reference_list_request=request)


## 3. Setup compliance rules

### 3.1 A simple PV contribution threshold check

In [81]:
pvInReportCcy = lm.AddressKeyComplianceParameter("passed_validation", compliance_parameter_type='AddressKeyComplianceParameter')
pvInReportCcy._value = "Valuation/PvInReportCcy"

upsert_compliance_rule_request = UpsertComplianceRuleRequest(
    id=lm.ResourceId(scope=scope, code='rule-1'),
    name='threshold-pv-check',
    description='Simple threshold PV check',
    template_id=lm.ResourceId(scope='system', code='PercentCheck'),
    variation='simple',
    portfolio_group_id=lm.ResourceId(scope=scope, code=portfolio_group_code),
    active=True,
    parameters={
        "Metric": pvInReportCcy,
        "UpperBound":    hf.decimal_parameter(10),
        "LowerBound":    hf.decimal_parameter(-1),
        "UpperWarning":  hf.decimal_parameter(30),
        "LowerWarning":  hf.decimal_parameter(-1),
        "GroupingPropertyKey":  hf.propertykey_parameter("Instrument/default/Isin")
    },
    properties={}
)

compliance_api.upsert_compliance_rule(upsert_compliance_rule_request=upsert_compliance_rule_request)

### 3.2 A restricted sector check

In [82]:
upsert_compliance_rule_request = UpsertComplianceRuleRequest(
    id=lm.ResourceId(scope=scope, code='rule-2'),
    name="restricted-sector-check",
    description='Simple sector restriction',
    template_id=lm.ResourceId(scope='system', code='PropertyValueNotInList'),
    variation='standard-not-in-list',
    portfolio_group_id=lm.ResourceId(scope=scope, code=portfolio_group_code),
    active=True,
    parameters={
        "PropertyKey":           hf.propertykey_parameter(f"Instrument/simpleCompliance/sector"),
        "ExclusivePropertyList": hf.stringlist_parameter(scope, "restricted-sectors")
    },
    properties={}
)

compliance_api.upsert_compliance_rule(upsert_compliance_rule_request=upsert_compliance_rule_request)

### 3.3 A security count limit check

In [83]:
upsert_compliance_rule_request = UpsertComplianceRuleRequest(
    id=lm.ResourceId(scope=scope, code='rule-3'),
    name="security-count-check",
    description='Simple security count restriction',
    template_id=lm.ResourceId(scope='system', code='NumberOfSecurities'),
    variation='standard',
    portfolio_group_id=lm.ResourceId(scope=scope, code=portfolio_group_code),
    active=True,
    parameters={
        "UpperBound":    hf.decimal_parameter(20),
        "LowerBound":    hf.decimal_parameter(-1),
        "UpperWarning":  hf.decimal_parameter(10),
        "LowerWarning":  hf.decimal_parameter(-1),
        "Excludes":      hf.portfolioidlist_parameter(scope, "empty-portfolio-list")
    },
    properties={}
)

compliance_api.upsert_compliance_rule(upsert_compliance_rule_request=upsert_compliance_rule_request)

## 4. Run Compliance Rules

Currently the API enables us to run compliance asat now, rather than for a specific date.

In [84]:
run_response = compliance_api.run_compliance(run_scope=scope,rule_scope=scope,is_pre_trade=True,recipe_id_scope='User',recipe_id_code='valuation_recipe_mid')
run_code = run_response.run_id.code

print(f"Compliance run {run_response.run_id.scope}/{run_response.run_id.code} completed.")
print(f"Instigated: {run_response.instigated_at}, completed: {run_response.completed_at}")

Compliance run simpleCompliance/14d5eb24-947d-4da1-9437-cd46e5b47a96 completed.
Instigated: 2024-01-29 17:24:17.092479+00:00, completed: 2024-01-29 17:24:17.545079+00:00


## 5. Analyse the compliance result

### 5.1 Identify rule-level failures

At a very coarse level we can determine which rules have been breached in this compliance run. We can also see a list of orders and portfolios affected by the breach.

The list of orders might well be wider than initially expected. This is because LUSID compliance makes no assumptions about which orders take precedence; it takes a holistic, contingent view about compliance taking into account all orders and existing positions.

Here you can see that `rule-1` has failed, and one portfolio is affected by the failure. We'll drilldown into this rule result in the following section.

In [85]:
run_summary = compliance_api.get_decorated_compliance_run_summary(scope=scope, code=run_code)

df = hf.rule_level_dataframe(run_summary)

print(f"Rule-level results for run {run_summary.run_id.scope}/{run_summary.run_id.code}.")
display(df)

Rule-level results for run simpleCompliance/14d5eb24-947d-4da1-9437-cd46e5b47a96.


\
                      Rule                   Rule Description   Status   
0  simpleCompliance/rule-2          Simple sector restriction   Failed   
1  simpleCompliance/rule-3  Simple security count restriction  Warning   
2  simpleCompliance/rule-1          Simple threshold PV check   Failed   

                                       
  Affected Orders Affected Portfolios  
0               0                   1  
1               0                   1  
2               0                   1

### 5.2 Drilldown into specific rule results

#### 5.2.1 Simple PV threshold breach

`rule-1` failed the compliance check overall. This rule is a simple 10% PV contribution threshold check often used to control portfolio diversification, so a failure indicates that one or more instruments contributed more than 10%. Taking a look at the drilldown data for this rule we can see that `GB00B1XZS820`, `GB00BH4HKS39`, `GB00BGDT3G23` and `GB00BH0P3Z91` in the `EQUITY_UK` portfolio each contribute more than 10%.

The rule is structured to
- start with all holdings in portfolio group `EQUITY_GROUP_UK` (the initial group)
- group contributions *by portfolio id*
- further group contributions for each portfolio *by Isin*
- finally, compare a pair of Results Used (Valuation/PVInReportCcy compared to portfolio-level Valuation/PVInReportCcy) *(checking that one is less than 10% of the other)*

We can see this structure represented in the template variation we've chosen for this rule, with the parameters each step requires:

In [92]:
percent_check = compliance_api.get_compliance_template(scope="system", code="PercentCheck")
steps = [v for v in percent_check.variations if v.label == 'simple'][0].steps

for s in steps:
    hf.variation_steps_to_dataframe(s)

Step Label    Step Type
0  Portfolios  GroupByStep

Params
      Step Label    Step Type GroupingPropertyKey
0  FinalGrouping  GroupByStep         PropertyKey

Limit Params                         \
  Step Label        Step Type   LowerBound      Metric UpperBound   
0    Compare  GroupFilterStep      Decimal  AddressKey    Decimal   

  Warning Params                           
    LowerWarning      Metric UpperWarning  
0        Decimal  AddressKey      Decimal

This information is represented by the Lineage for each drilldown row; the Lineage can be used to get a high-level view of what's caused a rule breach, and a pointer to where to start more detailed investigations if needed.

In [87]:

rule_result = compliance_api.get_compliance_rule_result(run_scope=scope, run_code=run_code, rule_scope=scope, rule_code='rule-1')

df = hf.rule_result_dataframe(rule_result)

print(f"Rule drilldown for {rule_result.rule_result.rule_id.scope}/{rule_result.rule_result.rule_id.code} from {rule_result.schedule} run {rule_result.run_id.scope}/{rule_result.run_id.code}, asat {rule_result.instigated_at}")
display(df[df['Details']['Status'] == 'Failed'])

Rule drilldown for simpleCompliance/rule-1 from PreTrade run simpleCompliance/14d5eb24-947d-4da1-9437-cd46e5b47a96, asat 2024-01-29 17:24:17.092479+00:00


Lineage                                                         Details  \
   Initial                  Portfolios FinalGrouping       Compare  Status   
6  Initial  simpleCompliance/EQUITY_UK  GB00BH0P3Z91  GB00BH0P3Z91  Failed   
7  Initial  simpleCompliance/EQUITY_UK  GB00BGDT3G23  GB00BGDT3G23  Failed   
8  Initial  simpleCompliance/EQUITY_UK  GB00BH4HKS39  GB00BH4HKS39  Failed   
9  Initial  simpleCompliance/EQUITY_UK  GB00B1XZS820  GB00B1XZS820  Failed   

                                     Results Used                          \
  Missing Data Portfolios.Valuation/PvInReportCcy Valuation/PvInReportCcy   
6            0                          3417890.0                772200.0   
7            0                          3417890.0                372000.0   
8            0                          3417890.0                490500.0   
9            0                          3417890.0                488950.0   

  Parameters Used             
       LowerBound UpperBound  
6              -1         10  
7              -1         10  
8              -1         10  
9              -1         10

### 5.2.2 Restricted sector breach

`rule-2` failed the compliance check overall. This rule prevents trade in a restricted set of sectors (specifically just Mining in this case), so a failure indicates that one or more holdings across the whole portfolio group are of an instrument from a restricted sector.

In [88]:
rule_result = compliance_api.get_compliance_rule_result(run_scope=scope, run_code=run_code, rule_scope=scope, rule_code='rule-2')

df = hf.rule_result_dataframe(rule_result)

print(f"Rule drilldown for {rule_result.rule_result.rule_id.scope}/{rule_result.rule_result.rule_id.code} from {rule_result.schedule} run {rule_result.run_id.scope}/{rule_result.run_id.code}, asat {rule_result.instigated_at}")
display(df[df['Details']['Status'] == 'Failed'])

Rule drilldown for simpleCompliance/rule-2 from PreTrade run simpleCompliance/14d5eb24-947d-4da1-9437-cd46e5b47a96, asat 2024-01-29 17:24:17.092479+00:00


Lineage                                             \
   Initial                            PropertyGrouped   
5  Initial  Instrument/simpleCompliance/sector=Mining   

                                             Details               \
                                     Compare  Status Missing Data   
5  Instrument/simpleCompliance/sector=Mining  Failed            0   

        Parameters Used                                      
  ExclusivePropertyList                         PropertyKey  
5                Mining  Instrument/simpleCompliance/sector

### 5.2.3 Security count warning

`rule-3` doesn't fail, but does trigger a warning. This rule is configured to warn if there are more than 20 securities in the portfolio.
TODO: confirm this is even working right.

In [89]:
rule_result = compliance_api.get_compliance_rule_result(run_scope=scope, run_code=run_code, rule_scope=scope, rule_code='rule-3')
df = hf.rule_result_dataframe(rule_result)

print(f"Rule drilldown for {rule_result.rule_result.rule_id.scope}/{rule_result.rule_result.rule_id.code} from {rule_result.schedule} run {rule_result.run_id.scope}/{rule_result.run_id.code}, asat {rule_result.instigated_at}")
display(df[df['Details']['Status'] == 'Warning'])

Rule drilldown for simpleCompliance/rule-3 from PreTrade run simpleCompliance/14d5eb24-947d-4da1-9437-cd46e5b47a96, asat 2024-01-29 17:24:17.092479+00:00


Lineage                             \
   Initial  WithoutExcludedPortfolios   
0  Initial  WithoutExcludedPortfolios   

                                              \
                                  Portfolios   
0  local-platform/simpleCompliance/EQUITY_UK   

                                              Details               \
                                  FinalCheck   Status Missing Data   
0  local-platform/simpleCompliance/EQUITY_UK  Warning            0   

  Parameters Used             
       LowerBound UpperBound  
0              -1         20